<center>

### COSC2753 - Machine Learning

# **Logistic Regression**

<center>────────────────────────────</center>
&nbsp;


# I. Global Configuration

In [219]:
import sys
import importlib
import tabulate
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import sklearn
import statsmodels
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.exceptions import FitFailedWarning
import warnings
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

# Reload modules
sys.path.append("../../")  # Root directory
modules_to_reload = [
    "scripts.styler",
    "scripts.neko",
    "scripts.utils",
    "scripts.outlier_detector",
]

# Reload modules if they have been modified
missing_modules = []

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    else:
        missing_modules.append(module_name)

# Recache missing modules
if missing_modules:
    print(f"Modules {missing_modules} not found. \nRecaching...")

# Import user-defined scripts
from scripts.styler import Styler
from scripts.neko import Neko
from scripts.utils import Utils
from scripts.outlier_detector import OutlierDetector

# Initialize styler
styler = Styler()  # Text Styler

# Check package versions
styler.draw_box("Checking Package Versions...")

try:
    with open("../../requirements.txt", "r") as file:
        requirements = file.readlines()
except FileNotFoundError:
    print(f"File '../../requirements.txt' not found.")

packages_to_check = [np, pd, sns, matplotlib, tabulate, sklearn, statsmodels, imblearn]

for package in packages_to_check:
    Utils.version_check(package, requirements=requirements)

styled_text = styler.style(
    "\nDone checking packages version...\n", bold=True, italic=True
)
print(styled_text)

# Initialize objects
styler.draw_box("Initializing Project...")
neko = Neko()  # Panda extension
bullet = ">>>"  # Bullet point
plt = matplotlib.pyplot  # Matplotlib

# Configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.precision", 3)

styled_text = styler.style("Done initializing project...", bold=True, italic=True)
print(styled_text)

┌────────────────────────────────┐
│  Checking Package Versions...  │
└────────────────────────────────┘
>>> numpy is up to date: 1.26.4
>>> pandas is up to date: 2.2.1
>>> seaborn is up to date: 0.13.2
>>> matplotlib is up to date: 3.8.3
>>> tabulate is up to date: 0.9.0
>>> sklearn is up to date: 1.4.1.post1
>>> statsmodels is up to date: 0.14.1
>>> imblearn is up to date: 0.12.2

Done checking packages version...

┌───────────────────────────┐
│  Initializing Project...  │
└───────────────────────────┘

    /\_____/\
   /  x   o  \
  ( ==  ^  == )       Neko has arrived!
   )         (        An data visualizing extension for analyzing DataFrames.
  (           )       Art: https://www.asciiart.eu/animals/cats.
 ( (  )   (  ) )
(__(__)___(__)__)

Done initializing project...


# II. Data Loading

In [220]:
try:
    # Load data
    df_train = pd.read_csv("../../data/processed/data_train_processed.csv")
    df_test = pd.read_csv("../../data/test/data_test.csv")

    styler.draw_box("Data Loaded Successfully")

except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
except Exception as e:
    print("An error occurred:", e)

┌────────────────────────────┐
│  Data Loaded Successfully  │
└────────────────────────────┘


In [221]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71262 entries, 0 to 71261
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HighBP                71262 non-null  int64  
 1   HighChol              71262 non-null  int64  
 2   CholCheck             71262 non-null  int64  
 3   BMI                   71262 non-null  int64  
 4   Smoker                71262 non-null  int64  
 5   Stroke                71262 non-null  int64  
 6   HeartDiseaseorAttack  71262 non-null  int64  
 7   PhysActivity          71262 non-null  int64  
 8   Fruits                71262 non-null  int64  
 9   Veggies               71262 non-null  int64  
 10  HvyAlcoholConsump     71262 non-null  int64  
 11  AnyHealthcare         71262 non-null  int64  
 12  NoDocbcCost           71262 non-null  int64  
 13  GenHlth               71262 non-null  int64  
 14  MentHlth              71262 non-null  int64  
 15  PhysHlth           

In [222]:
df_train.head(10)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,ExtraMedTest,ExtraAlcoholTest,Status
0,0,0,1,22,0,0,0,1,1,1,0,1,0,1,0,0,0,0,6,6,8,-7.416,-7.566,0
1,0,0,1,21,0,0,0,1,1,1,0,1,0,1,0,0,0,0,6,6,8,-7.416,-7.566,0
2,0,0,1,22,0,0,0,1,1,1,0,1,0,1,0,0,0,0,6,6,8,-7.416,-7.566,0
3,0,0,1,21,0,0,0,1,1,1,0,1,0,1,0,0,0,0,6,6,8,-7.416,-7.566,0
4,0,0,1,22,1,0,0,1,1,1,0,1,0,1,0,0,0,0,6,6,8,-7.416,-7.566,0
5,1,0,1,29,0,0,0,1,1,1,0,1,0,2,0,0,0,1,9,6,8,-7.416,-7.566,0
6,0,0,1,21,0,0,0,1,1,1,0,1,0,1,0,0,0,0,3,6,8,-7.416,-7.566,0
7,0,0,1,22,0,0,0,1,1,1,0,1,0,1,0,0,0,0,7,6,8,-7.416,-7.566,0
8,1,0,1,31,0,0,0,1,1,1,0,1,0,2,0,0,0,1,11,6,8,-7.416,-7.566,0
9,0,0,1,20,0,0,0,1,1,1,0,1,0,1,0,0,0,0,6,6,8,-7.416,-7.566,0


# III. Model Development

## 1. Feature Scaling

### Scaling Techniques for Logistic Regression Models

Logistic regression models often benefit from scaled features to enhance accuracy. When dealing with varied range numerical data, two common scaling techniques are **standardization** (using a *StandardScaler*) and **Normalisation** (using a *MinMaxScaler*).

**Limitations of Log Transformation and StandardScaler:** 
The *StandardScaler*, which typically involves log transformation followed by standardization, might not be suitable for this specific case. This method can encounter issues when the data contains zeros or negative values, potentially leading to inaccurate transformations.

Therefore, the *MinMaxScaler* is a more appropriate choice for this scenario. This technique effectively handles skewed data by rescaling each feature to a specific range (often 0 to 1). This ensures that all features contribute equally to the logistic regression model, improving its ability to learn the underlying relationships between features and the target variable.

In [223]:
# Scale features using min-max scaler (Normalization)
df_train = neko.scale_feature(df_train, "norm")

df_train.describe()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,ExtraMedTest,ExtraAlcoholTest,Status
count,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.000,71262.0
mean,0.522,0.503,0.974,0.190,0.452,0.050,0.125,0.745,0.636,0.809,0.044,0.964,0.070,0.402,0.076,0.129,0.192,0.471,0.645,0.805,0.723,0.594,0.589,0.5
std,0.500,0.500,0.160,0.074,0.498,0.218,0.330,0.436,0.481,0.393,0.206,0.186,0.255,0.270,0.219,0.288,0.394,0.499,0.221,0.198,0.290,0.247,0.250,0.5
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0
25%,0.000,0.000,1.000,0.141,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.250,0.000,0.000,0.000,0.000,0.500,0.600,0.571,0.463,0.462,0.0
50%,1.000,1.000,1.000,0.176,0.000,0.000,0.000,1.000,1.000,1.000,0.000,1.000,0.000,0.500,0.000,0.000,0.000,0.000,0.667,0.800,0.857,0.500,0.500,0.5
75%,1.000,1.000,1.000,0.224,1.000,0.000,0.000,1.000,1.000,1.000,0.000,1.000,0.000,0.500,0.000,0.067,0.000,1.000,0.833,1.000,1.000,0.805,0.795,1.0
max,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0


## 2. Model Training

In [224]:
# Split the data into training and testing sets
X = df_train.drop(columns=["Status"], axis=1)
y = df_train["Status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### 2.1 Model Evaluation (First Look)

Evaluation within the data preprocessing notebook indicates that the features **AnyHealthcare** and **MentHlth** do not exhibit a statistically significant correlation with the target variable **Status**. Consequently, these two features will be excluded from the dataset, and the model will be retrained to assess the impact of this refinement.

In [225]:
# Traing the model (With All Features)
styler.draw_box("Training the model (With All Features)")
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Classification report
report = classification_report(y_test, y_pred)
print(report)

# Traing the model (With reduced Features)
styler.draw_box("Training the model (With Reduced Features)")
model = LogisticRegression(max_iter=1000, random_state=42)

# Drop the specified columns from X_train and X_test
X_train_reduced = X_train.drop(columns=["AnyHealthcare", "MentHlth"])
X_test_reduced = X_test.drop(columns=["AnyHealthcare", "MentHlth"])

model.fit(X_train_reduced, y_train)

# Make predictions
y_pred = model.predict(X_test_reduced)

# Classification report
report = classification_report(y_test, y_pred)
print(report)

┌──────────────────────────────────────────┐
│  Training the model (With All Features)  │
└──────────────────────────────────────────┘
              precision    recall  f1-score   support

         0.0       0.78      0.84      0.81      7158
         1.0       0.83      0.76      0.79      7095

    accuracy                           0.80     14253
   macro avg       0.81      0.80      0.80     14253
weighted avg       0.81      0.80      0.80     14253

┌──────────────────────────────────────────────┐
│  Training the model (With Reduced Features)  │
└──────────────────────────────────────────────┘
              precision    recall  f1-score   support

         0.0       0.78      0.83      0.80      7158
         1.0       0.82      0.76      0.79      7095

    accuracy                           0.80     14253
   macro avg       0.80      0.80      0.80     14253
weighted avg       0.80      0.80      0.80     14253



**Evaluation Metrics after Feature Selection**

After applying feature selection, the evaluation metrics reveal a decline in model accuracy. This suggests that the removed features may have contained information critical to the model's performance.

→ *Therefore, to ensure optimal performance, all features in the current dataset will be retained for further analysis.*

### 2.2 Feature Selection

This section delves deeper into feature selection by employing a **Wrapper Method** known as **Recursive Feature Elimination with Cross-Validation (REFCV)**. This technique iteratively evaluates the performance of a chosen machine learning model on progressively smaller subsets of features. Features deemed least impactful on the model's performance are eliminated in each step. The RECFCV process continues until the optimal feature combination is identified, resulting in the model achieving its highest accuracy.

While sequential feature selection methods, such as forward selection or backward selection, might be considered for this task, they are often susceptible to instability and prone to overfitting. Due to these potential drawbacks, this approach is not employed in this instance. RECFCV, on the other hand, offers a more robust and reliable method for identifying the optimal feature subset that maximizes model performance.


In [226]:
styler.draw_box("Feature Selection (Wrapper Method - Recursive Feature Elimination with Cross-Validation (RFECV))")

# Create Logistic Regression model
logistic_model = LogisticRegression(random_state=42, max_iter=1000)

# Create RFECV with Logistic Regression model
selector = RFECV(estimator=logistic_model, cv=5, scoring="accuracy")
selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[selector.support_]

print("Selected features using Logistic Regression and RFECV:", selected_features)
print("Size of selected features:", len(selected_features))

┌────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  Feature Selection (Wrapper Method - Recursive Feature Elimination with Cross-Validation (RFECV))  │
└────────────────────────────────────────────────────────────────────────────────────────────────────┘
Selected features using Logistic Regression and RFECV: Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Stroke',
       'HeartDiseaseorAttack', 'HvyAlcoholConsump', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'ExtraMedTest', 'ExtraAlcoholTest'],
      dtype='object')
Size of selected features: 18


### 2.3 Model Evaluation (Second Look)

In [227]:
# Traing the model
styler.draw_box("Training the model (Without Hyperparameter Tuning)")
model = LogisticRegression(max_iter=1000, random_state=42)

neko.evaluate_model(
    model, X_train[selected_features], y_train, X_test[selected_features], y_test
)

┌──────────────────────────────────────────────────────┐
│  Training the model (Without Hyperparameter Tuning)  │
└──────────────────────────────────────────────────────┘
Classification Report for Training Data:
               precision    recall  f1-score   support

         0.0       0.79      0.84      0.81     28473
         1.0       0.83      0.77      0.80     28536

    accuracy                           0.81     57009
   macro avg       0.81      0.81      0.81     57009
weighted avg       0.81      0.81      0.81     57009

Classification Report for Testing Data:
               precision    recall  f1-score   support

         0.0       0.78      0.84      0.81      7158
         1.0       0.83      0.76      0.79      7095

    accuracy                           0.80     14253
   macro avg       0.80      0.80      0.80     14253
weighted avg       0.80      0.80      0.80     14253



The **feature selection process** identified a significant result. Only **18 features** were ultimately chosen for the prediction model. This reduced feature set will be used for training due to the observed increase in accuracy, rising from **80% to 81%**. This decision is further bolstered by improvements in both **accuracy** and **F1-score**, which are crucial metrics for evaluating model performance.

Furthermore, the high degree of similarity between **accuracy** and **F1-score** on both the training and testing datasets suggests **strong model generalizability**. In simpler terms, the model performs well not only on the data it was trained on but also on unseen data. This signifies the model's ability to learn underlying patterns and make accurate predictions on new information.


### 2.4 Outlier Detection and Treatment

In [228]:
# Initialize outlier detector
outlier_detector = OutlierDetector()

# Specify non-binary numerical columns
numerical_columns = ["MentHlth", "PhysHlth", "BMI", "ExtraMedTest", "ExtraAlcoholTest"]

# Create a copy of the DataFrame (for comparison purposes)
df_tmp = df_train.copy()

# Detect and handle outliers
for column in numerical_columns:
    # Detect outliers using IQR method
    outliers = outlier_detector.find_outliers_iqr(df_train[column], threshold=1.5)

    # Show outliers results
    print(f"Outliers for {column}:")
    print(outliers["table"])

    # Handle outliers (Winsorization)
    print(">>> Resolving outliers...")
    if outliers["total_outliers"] > 0:
        # Get the lower and upper bounds
        lower_bound = outliers["lower_bound"]
        upper_bound = outliers["upper_bound"]

        # Winsorize the outliers
        df_tmp[column] = np.clip(df_train[column], lower_bound, upper_bound)

    print(f">>> Outliers for {column} resolved.\n")

Outliers for MentHlth:
╭─────────────────┬─────────────────────────────────────────────────────────────────────────────╮
│ Key             │ Value                                                                       │
├─────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│ Unique Outliers │ 0.23333333333333334, 1.0, 0.3333333333333333, 0.2, 0.26666666666666666, ... │
├─────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│ Lower Bound     │ 0.0                                                                         │
├─────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│ Upper Bound     │ 0.0                                                                         │
├─────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│ Threshold       │ 1.5                                                                        

### 2.5 Model Evaluation (Third Look)

In [229]:
# Split the data into training and testing sets
X_tmp = df_tmp.drop(columns=["Status"], axis=1)
y_tmp = df_tmp["Status"]

X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(
    X_tmp, y_tmp, test_size=0.2, random_state=42
)

# Traing the model
styler.draw_box("Training the model")
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tmp, y_train_tmp)

neko.evaluate_model(model, X_train_tmp, y_train_tmp, X_test_tmp, y_test_tmp)

┌──────────────────────┐
│  Training the model  │
└──────────────────────┘


Classification Report for Training Data:
               precision    recall  f1-score   support

         0.0       0.78      0.83      0.80     28473
         1.0       0.82      0.77      0.79     28536

    accuracy                           0.80     57009
   macro avg       0.80      0.80      0.80     57009
weighted avg       0.80      0.80      0.80     57009

Classification Report for Testing Data:
               precision    recall  f1-score   support

         0.0       0.78      0.83      0.80      7158
         1.0       0.82      0.76      0.79      7095

    accuracy                           0.80     14253
   macro avg       0.80      0.80      0.80     14253
weighted avg       0.80      0.80      0.80     14253



The **removal of outliers** from the data resulted in a **1% decrease** in model accuracy. This suggests that the outliers may have contained relevant information that contributed to the model's performance. Consequently, the model will be **retrained using the complete original dataset** to achieve optimal performance.

### 2.4 Model Tuning (Hyperparameter Optimization)

In [232]:
# Warning suppression
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Hyperparameter Tuning
param_grid = {
    "penalty": ["l1", "l2", "elasticnet", None],
    "C": np.logspace(-3, 3, 20),
    "solver": ["lbfgs", "liblinear", "sag", "saga"],
    "max_iter": [1000, 2500, 5000],
    "l1_ratio": np.linspace(0, 1, 10),
}

Based on the analysis, there is a strong likelihood of achieving an optimal or near-optimal solution by setting the number of iterations to `60`. To further improve the probability of success, I propose increasing the number of iterations (n_iter) to `100`. This adjustment will allow the model to explore a broader range of hyperparameters, potentially identifying an even more optimal solution.

https://web.archive.org/web/20160701182750/http://blog.dato.com/how-to-evaluate-machine-learning-models-part-4-hyperparameter-tuning

In [233]:
# Polynomial degrees
degrees = range(1, 3)  # 1 to 2 (because of computational cost for higher degrees)

# Store best models
best_models = {}

# Loop through each degree
for degree in degrees:
    print(">>> Hyperparameter Tuning For Polynomial Degree:", degree)

    poly_features = PolynomialFeatures(degree=degree)
    X_train_poly = poly_features.fit_transform(X_train[selected_features])
    X_test_poly = poly_features.transform(X_test[selected_features])

    # Grid search
    logistic_regression = LogisticRegression(random_state=42)

    search = HalvingGridSearchCV(
        logistic_regression,
        param_grid=param_grid,
        cv=10,
        scoring="f1_weighted",
        n_jobs=2,
    )
    search.fit(X_train_poly, y_train)

    # Store the best model and its parameters
    best_models[degree] = {
        "best_model": search.best_estimator_,
        "best_params": search.best_params_,
    }

    styler.draw_box(f"Best parameters for degree {degree}")
    print(search.best_params_)

    print("Best accuracy:", search.best_score_)

>>> Hyperparameter Tuning For Polynomial Degree: 1
┌────────────────────────────────┐
│  Best parameters for degree 1  │
└────────────────────────────────┘
{'C': 2.976351441631316, 'l1_ratio': 0.5555555555555556, 'max_iter': 2500, 'penalty': 'l2', 'solver': 'lbfgs'}
Best accuracy: 0.8071532450861929
>>> Hyperparameter Tuning For Polynomial Degree: 2
┌────────────────────────────────┐
│  Best parameters for degree 2  │
└────────────────────────────────┘
{'C': 1.438449888287663, 'l1_ratio': 0.7777777777777777, 'max_iter': 5000, 'penalty': 'l1', 'solver': 'liblinear'}
Best accuracy: 0.8289773622459047
